In [4]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import os

In [1]:
# models = ['densenet201', 'densenet121', 'resnet50', 'resnet101', 'se_resnet50']
models = ['resnet34', 'resnet50']

In [ ]:
dfs = []
submissions = [f'model_{model}_fold_{fold}/submission.csv' for model in models for fold in range(5)]

n_submissions = len(submissions)

for submission in submissions:
    dfs.append(pd.read_csv(submission))

In [ ]:
dfs[0].head()

In [ ]:
df = pd.DataFrame(columns=dfs[0].columns)

In [ ]:
dfs[0].shape[0]

In [ ]:
for i in range(dfs[0].shape[0]):
    attributes = np.array([])
    id = dfs[0].loc[i, 'id']
    attributes = np.concatenate([submission.loc[i, 'attribute_ids'].split(' ') for submission in dfs])
    unique, counts = np.unique(attributes, return_counts=True)
    attribute_ids = ' '.join(unique[counts > n_submissions // 3])
    data = [id, attribute_ids]
    tmp_df = pd.DataFrame([data], columns=df.columns)
    df = pd.concat([df, tmp_df])
    

In [ ]:
df.head()

In [ ]:
df.to_csv('submission_ensemble_voting_3.csv', index=False)

In [ ]:
df.shape

In [ ]:
pd.read_csv('submission_ensemble_voting.csv').head()

In [2]:
from sklearn.multioutput import MultiOutputRegressor
import gc

In [8]:
X = np.empty((1103, 2))
y_true = np.empty(1103)

In [5]:
y = pd.read_csv('folds.csv')

In [ ]:
for fold in range(5):
    X_x = []
    for model in models:
        model_path = os.path.join(f'model_{model}_fold_{fold}', f'val_{fold}.h5')
        X_tmp = pd.read_hdf(model_path)
        X_x = np.hstack((X_x, X_tmp.values))
    X = np.vstack()
    y_tmp = np.zeros((X_tmp.shape[0], 1103))
    masks = [[int(c) for c in s.split(' ')] for s in y[y['fold'] == fold]['attribute_ids'].values]
    print(y_tmp.shape[0], len(masks))
    for i in range(y_tmp.shape[0]):
        y_tmp[i, masks[i]] = 1
    y_true = np.vstack((y_true, y_tmp))
    del X_tmp, y_tmp, masks
    gc.collect()    


In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
model = MultiOutputRegressor(Ridge(alpha=6), n_jobs=1)

In [ ]:
X = X[1:]
y_true = y_true[1:]

In [ ]:
model.fit(X, y_true)